In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

In [ ]:

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = models.vgg19(weights=None)
model.classifier[6] = nn.Linear(4096, 5)  
model.load_state_dict(torch.load('model_vgg19_adverse_env.pth', map_location=device))
model.eval().to(device)

In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

In [ ]:

image_path = 'so.bmp'  
image = Image.open(image_path).convert('RGB')
image = transform(image).unsqueeze(0).to(device)  

# Forward pass through the model
with torch.no_grad():
    outputs = model(image)
    probabilities = torch.softmax(outputs, dim=1) 
    predicted = torch.argmax(probabilities, dim=1) 

# Print out the prediction
class_names = ['Haze', 'Lens Blur', 'Rain', 'Shadow', 'Snow']  
predicted_class = class_names[predicted.item()]
print(f"Predicted class for input image: {predicted_class}")